# Baseline model and Blender communication

In [8]:
import socket
import time
import math
import sys
import numpy as np
import cv2


In [ ]:

    def _draw_tires(self, screen):
        angle = -self.heading.angle_to(Vector2(1, 0))
        tire_width = 0.1
        tire_length = 0.3
        tire_corner_points_back = [
            Vector2(-tire_length / 2, tire_width / 2),
            Vector2(tire_length / 2, tire_width / 2),
            Vector2(tire_length / 2, -tire_width / 2),
            Vector2(-tire_length / 2, -tire_width / 2)
        ]
        tire_corner_points_front = [
            Vector2(-tire_length / 2, tire_width / 2),
            Vector2(tire_length / 2, tire_width / 2),
            Vector2(tire_length / 2, -tire_width / 2),
            Vector2(-tire_length / 2, -tire_width / 2)
        ]
        tire_corner_points = [p.rotate(angle + self.steering) for p in tire_corner_points]
        tire_corner_points = [p * self.ppu for p in tire_corner_points]
        
        tires = [
            Vector2(-self.length / 2, self.width / 2),
            Vector2(self.length / 2, self.width / 2),
            Vector2(self.length / 2, -self.width / 2),
            Vector2(-self.length / 2, -self.width / 2)
        ]

        # rotate front tires
        tires = [p.rotate(angle) for p in tires]
        tires = [p * self.ppu for p in tires]
        tires = [p + self.position for p in tires]


        for tire in tires:
            points = [p + tire for p in tire_corner_points]
            pygame.draw.polygon(screen, (255, 255, 255), points, 1)



In [8]:
import sys
import numpy as np
import cv2
import logging
import os
import math
import pygame
from pygame.math import Vector2



class Car:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=30, max_acceleration=30.0):
        self.position = Vector2(x, y)
        self.heading = Vector2(1, 0)
        self.velocity = Vector2(0.0, 0.0)
        self.length = length
        self.max_steering = max_steering
        self.steering = 0.0 # degrees


    def update(self, dt):
        ...

    def update_inputs(self, dt):
        pressed = pygame.key.get_pressed()

        steering_speed = 10
        if pressed[pygame.K_LEFT]:
            self.steering = max(-self.max_steering, self.steering - steering_speed * dt)

        if pressed[pygame.K_RIGHT]:
            self.steering += min(self.max_steering, self.steering + steering_speed * dt)
        
        print(self.steering)


class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Slamcar Controller")
        width = 1280 // 2
        height = 720 // 2
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):

        car = Car(0, 0)
        pixel_per_unit = 32 

        while not self.exit:
            dt = self.clock.get_time() / 100

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # User input
            car.update_inputs(dt)


            self.clock.tick(self.ticks)
        pygame.quit()


game = Game()
game.run()

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [39]:
from IPython.display import clear_output
import keyboard


def send_packet(position, theta):
    packet = f'{position[0]},{position[1]},{0},{theta}'.encode('utf-8')
    sock.sendall(packet)


position = np.array([0, 0], dtype=np.float32)
velocity = np.array([0, 0], dtype=np.float32)
theta = 0


while not keyboard.is_pressed('q'):

    if keyboard.is_pressed('w'):
        velocity += np.array([math.cos(theta/180*math.pi), math.sin(theta/180*math.pi)], dtype=np.float32)
    if keyboard.is_pressed('s'):
        velocity -= np.array([math.cos(theta/180*math.pi), math.sin(theta/180*math.pi)], dtype=np.float32)
    if keyboard.is_pressed('a'):
        theta += 1
    if keyboard.is_pressed('d'):
        theta -= 1

    position += velocity * 0.01
    velocity *= 0.9

    clear_output(wait=True)
    print(f'Position: {position}, Velocity: {velocity}, Theta: {theta}')

    #cv2.imshow('image', np.zeros((100, 100, 3)))
    send_packet(position, theta)
    time.sleep(0.01)

cv2.destroyAllWindows() 

Position: [ 1.8579344 20.74566  ], Velocity: [1.0969788e-08 1.2538521e-07], Theta: 265


ConnectionResetError: [WinError 10054] Eine vorhandene Verbindung wurde vom Remotehost geschlossen

In [ ]:


cv2.destroyAllWindows()
# Clean up
packet = b'quit'
sock.sendall(packet)
sock.close()
print('Connection closed.')

# worker communication

In [1]:

import socket
import os
import tqdm

# device's IP address
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 5001

# receive 4096 bytes each time
BUFFER_SIZE = 4096
SEPARATOR = "<SEPARATOR>"

s = socket.socket()

# bind the socket to our local address
s.bind((SERVER_HOST, SERVER_PORT))

s.listen(5)
print(f"[*] Listening as {SERVER_HOST}:{SERVER_PORT}")

# accept connection if there is any
client_socket, address = s.accept() 
# if below code is executed, that means the sender is connected
print(f"[+] {address} is connected.")

# receive the file infos
# receive using client socket, not server socket
received = client_socket.recv(BUFFER_SIZE).decode()
filename, filesize = received.split(SEPARATOR)
# remove absolute path if there is
filename = os.path.basename(filename)
# convert to integer
filesize = int(filesize)

# start receiving the file from the socket
# and writing to the file stream
progress = tqdm.tqdm(range(filesize), f"Receiving {filename}", unit="B", unit_scale=True, unit_divisor=1024)
with open(filename, "wb") as f:
    while True:
        # read 1024 bytes from the socket (receive)
        bytes_read = client_socket.recv(BUFFER_SIZE)
        if not bytes_read:    
            # nothing is received
            # file transmitting is done
            break
        # write to the file the bytes we just received
        f.write(bytes_read)
        # update the progress bar
        progress.update(len(bytes_read))

# close the client socket
client_socket.close()
# close the server socket
s.close()

[*] Listening as 0.0.0.0:5001
[+] ('10.0.0.50', 42412) is connected.


Receiving test.jpg:   0%|          | 0.00/128k [00:00<?, ?B/s]